In [50]:
# import necessary libraries

from transformers import AutoModelForSeq2SeqLM
from transformers import AutoTokenizer
from transformers import GenerationConfig
from nltk.translate.bleu_score import sentence_bleu, corpus_bleu
import re

from googletrans import Translator  # First we need to !pip install googletrans

In [2]:
# Load model
model_name='facebook/mbart-large-50-many-to-many-mmt'
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

In [13]:
# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)
tokenizer.src_lang = "en_XX"
# tokenizer.tgt_lang = "fa_XX"

In [24]:
# Source sentences
source_sentences = {
    1: {"en": "I woke up early this morning.", "fa": "من امروز صبح زود بیدار شدم."},
    2: {"en": "She is reading a very interesting book.", "fa": "او دارد یک کتاب بسیار جالب می‌خواند."},
    3: {"en": "They went to the park to play football.", "fa": "آن‌ها برای بازی فوتبال به پارک رفتند."},
    4: {"en": "We had dinner at a nice restaurant last night.", "fa": "دیشب در یک رستوران خوب شام خوردیم."},
    5: {"en": "He doesn't like watching horror movies.", "fa": "او تماشای فیلم‌های ترسناک را دوست ندارد."},
    6: {"en": "Can you help me with this math problem?", "fa": "می‌تونی تو حل این مسئله ریاضی بهم کمک کنی؟"},
    7: {"en": "The weather is getting colder every day.", "fa": "هوا هر روز سردتر می‌شود."},
    8: {"en": "I have never been to Paris.", "fa": "من هرگز به پاریس نرفته‌ام."},
    9: {"en": "She always forgets where she puts her keys.", "fa": "او همیشه فراموش می‌کند کلیدهایش را کجا گذاشته."},
    10: {"en": "We are planning a trip to the mountains.", "fa": "ما داریم یک سفر به کوهستان برنامه‌ریزی می‌کنیم."},
}


In [73]:
# Function to get model output for a given prompt
def model_output(prompt):

  inputs = tokenizer(prompt, return_tensors='pt')
  generated_tokens = model.generate(**inputs, forced_bos_token_id=tokenizer.lang_code_to_id["fa_IR"], max_new_tokens=50)
  output = tokenizer.decode(generated_tokens[0], skip_special_tokens=True)

  return output

In [39]:
# Function to modify text by removing unwanted characters
def modify_text(text):
    text = re.sub(r"\bمی\s+", "می", text)
    text = text.replace("\u200c", "")
    return text

In [94]:
# Initialize lists to store predictions, references, and BLEU scores
predictions = []
refs = []
leu_scores_model = []

# Calculate BLEU scores for each sentence
for key, src in source_sentences.items():
  prompt = src['en']
  reference = src['fa']
  reference = modify_text(reference)
  reference = [reference.split()]

  output = model_output(prompt)

  output = modify_text(output)
  output = output.split()

  leu_score = sentence_bleu(reference, output)
  leu_scores_model.append(leu_score)
  print(f'leu_score for sample {key} is {leu_score:0.4f}.')

  refs.append(reference)
  predictions.append(output)

# Calculate corpus BLEU score
leu_score_model = corpus_bleu(refs, predictions)
print(f'leu_score_model for dataset is {leu_score_model:0.4f}.')

# **We can pass all sentences at once, as well.**

In [97]:
# Prepare prompts and references for generation
prompts = []
refs = []

for key, src in source_sentences.items():
  prompt = src['en']
  prompts.append(prompt)

  reference = src['fa']
  reference = modify_text(reference)
  reference = [reference.split()]
  refs.append(reference)

# Generate outputs using the model
inputs = tokenizer(prompts, return_tensors='pt', padding=True)
generated_tokens = model.generate(**inputs, forced_bos_token_id=tokenizer.lang_code_to_id["fa_IR"], max_new_tokens=50)


In [101]:
# Modify generated outputs and prepare for BLEU score calculation
predictions = []

for tr in generated_tokens:
  translate = tokenizer.decode(tr, skip_special_tokens=True)
  translate = modify_text(translate)
  translate = translate.split()

  predictions.append(translate)

# Calculate corpus BLEU score
leu_score_model = corpus_bleu(refs, predictions)
print(f'leu_score_model for dataset is {leu_score_model:0.4f}.')

# **Try Google Translate**

In [76]:
# Initialize the translator
translator = Translator()

In [83]:
# Prepare prompts and references for translation
predictions = []
refs = []
leu_scores_google = []

for key, src in source_sentences.items():
  prompt = src['en']
  reference = src['fa']
  reference = modify_text(reference)
  reference = [reference.split()]

  translated_result = await translator.translate(prompt, dest='fa')

  translated_result = modify_text(translated_result.text)
  translated_result = translated_result.split()

  leu_score = sentence_bleu(reference, translated_result)
  print(f'leu_score_google for sample {key} is {leu_score:0.4f}.')

  leu_scores_google.append(leu_score)

  refs.append(reference)
  predictions.append(translated_result)

# Calculate corpus BLEU score for Google Translate
leu_score_google = corpus_bleu(refs, predictions)
print(f'leu_score for dataset is {leu_score_google:0.4f}.')


# **Results**

In [95]:
# Compare BLEU scores between model and Google Translate
for i, (m, g) in enumerate(zip(leu_scores_model, leu_scores_google), 1):
    better = "Model" if m > g else "Google Translate" if g > m else "Tie"
    print(f"Sentence {i}: Model BLEU={m:.4f}, Google BLEU={g:.4f} → Better: {better}")

    if i == len(leu_scores_google):
      better = "Model" if leu_score_model > leu_score_google else "Google Translate" if leu_score_google > leu_score_model else "Tie"
      print(f"Overall: Model BLEU={leu_score_model:.4f}, Google BLEU={leu_score_google:.4f} → Better: {better}")

